In [7]:
import os
import re
import docx
import pandas as pd
from collections import defaultdict

# Словарь для преобразования текстовых чисел в числовые
TEXT_NUMBERS = {
    'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5, 'six': 6,
    'seven': 7, 'eight': 8, 'nine': 9, 'ten': 10, 'eleven': 11, 'twelve': 12,
    'thirteen': 13, 'fourteen': 14, 'fifteen': 15, 'sixteen': 16,
    'seventeen': 17, 'eighteen': 18, 'nineteen': 19, 'twenty': 20,
    'thirty': 30, 'forty': 40, 'fifty': 50, 'sixty': 60, 'seventy': 70,
    'eighty': 80, 'ninety': 90, 'hundred': 100
}

def word_to_number(word):
    """Преобразует текстовое число в числовое значение."""
    words = re.split(r'[\s\-]+', word.lower())  # Разбиваем по пробелам и дефисам
    number = 0
    temp = 0

    for w in words:
        if w in TEXT_NUMBERS:
            scale = TEXT_NUMBERS[w]
            if scale == 100:  # Обработка сотен, например "one hundred"
                temp *= scale
            else:
                temp += scale
        else:
            return None  # Если слово не распознано, возвращаем None

    number += temp
    return number

def extract_chapter_number(text):
    """Ищет и извлекает номер главы из текста."""
    # Проверяем числовой формат, например, "Chapter 1"
    numeric_match = re.match(r'Chapter\s+(\d+)', text, re.IGNORECASE)
    if numeric_match:
        return int(numeric_match.group(1))
    
    # Проверяем текстовый формат, например, "Chapter One" или "Chapter Thirty-One"
    text_match = re.match(r'Chapter\s+([\w\s\-]+)', text, re.IGNORECASE)
    if text_match:
        word = text_match.group(1).strip()
        return word_to_number(word)
    return None

def extract_text_from_docx(file_path):
    """Извлекает текст из .docx файла."""
    try:
        print(f"Пытаюсь открыть файл: {file_path}")
        doc = docx.Document(file_path)
        print("Файл успешно открыт.")
        full_text = []
        for paragraph in doc.paragraphs:
            full_text.append(paragraph.text)
        print("Текст успешно извлечён.")
        return '\n'.join(full_text)
    except Exception as e:
        print(f"Ошибка при открытии или чтении файла: {e}")
        return None

def split_text_into_chapters(text):
    """Разделяет текст на главы."""
    if not text:
        print("Текст пуст, невозможно разделить на главы.")
        return {}

    print("Разделяю текст на главы...")
    chapters = defaultdict(tuple)
    chapter_matches = list(re.finditer(r'Chapter\s+([\w\s\-]+)', text, re.IGNORECASE))

    if not chapter_matches:
        print("Не удалось найти главы.")
        return {}

    for i, match in enumerate(chapter_matches):
        start = match.start()
        chapter_title = match.group(0)
        chapter_number = extract_chapter_number(chapter_title)  # Извлекаем номер главы
        end = chapter_matches[i + 1].start() if i + 1 < len(chapter_matches) else len(text)
        chapter_text = text[start:end].strip()
        if chapter_number is not None:
            chapters[chapter_number] = (chapter_title, chapter_text)
    
    print(f"Текст успешно разделён на {len(chapters)} глав.")
    return chapters

# Функция для поиска слов в каждой главе
def search_words_in_chapter(chapter_text, words):
    word_pattern = r'\b(' + '|'.join(re.escape(word) for word in words) + r')\b'
    
    matches = []
    for match in re.finditer(word_pattern, chapter_text, re.IGNORECASE):
        start = match.start()
        end = match.end()
        
        # Берем по три слова до и после
        start_context = chapter_text[:start].split()[-8:]
        end_context = chapter_text[end:].split()[:8]
        
        # Получаем найденное слово и позицию в тексте
        found_word = chapter_text[start:end]
        result = ' '.join(start_context + [found_word] + end_context)
        
        matches.append((start, result))  # Сохраняем позицию и результат
    
    # Сортируем совпадения по позиции в тексте
    matches.sort(key=lambda x: x[0])
    
    return [match[1] for match in matches]
    
def search_in_all_chapters(chapters, words, category):
    if not chapters:
        print("Нет глав для поиска.")
        return []
    
    print(f"Ищу заданные слова для категории {category}...")
    results = []
    for chapter_number, (chapter_title, chapter_text) in chapters.items():
        matches = search_words_in_chapter(chapter_text, words)
        if matches:
            for match in matches:
                results.append([chapter_number, chapter_title, match, category])
    print(f"Найдено совпадений в {len(results)} главах для категории {category}.")
    return results

# Остальная часть кода остаётся прежней...

def main(docx_file_path, chapters_folder):
    if not os.path.exists(docx_file_path):
        print(f"Файл не найден: {docx_file_path}")
        return

    print(f"Извлекаю текст из {docx_file_path}...")
    book_text = extract_text_from_docx(docx_file_path)
    
    if not book_text:
        print("Ошибка: текст не был извлечён.")
        return

    chapters = split_text_into_chapters(book_text)
    
    if not chapters:
        print("Ошибка: главы не были найдены.")
        return

    clothes_words = ['stilettos', 'sunglasses', 'ring', 'necklace', 'bracelet', 'earrings', 'brooch', 'watch', 'anklet', 'choker', 'pendant', 'cufflinks', 'tie clip', 'nose ring', 'belly button ring', 'toe ring', 'hairpin', 'tiara', 'diadem', 'bangle', 'chain', 'medallion', 'pearl necklace', 'locket', 'armband', 'charm bracelet', 'dress', 'robe', 'suit', 'clothes', 'coat', 'jacket', 'shirt', 'pants', 'skirt', 'jeans', 't-shirt', 'sweater', 'blouse', 'shorts', 'hoodie', 'vest', 'scarf', 'hat', 'gloves', 'boots', 'shoes', 'sneakers', 'socks', 'tie', 'belt', 'gown', 'trench coat', 'blazer', 'cardigan', 'overalls', 'tank top', 'leggings']
    hair_words = ['hair', 'beard', 'ponytail', 'bun', 'braids', 'bob', 'pixie cut', 'long waves', 'curly hair', 'straight hair', 'afro', 'buzz cut', 'french twist', 'dreadlocks', 'fishtail braid', 'half-up half-down', 'side part', 'middle part', 'updo', 'loose curls', 'locks', 'layered cut', 'shag cut', 'crew cut', 'mohawk', 'bangs', 'chignon', 'top knot']
    appearances_words = ['fur', 'black wolf', 'white wolf', 'brown wolf', 'caramel hair', 'blonde', 'brunette', 'redhead', 'white hair', 'red hair', 'auburn hair', 'chestnut hair', 'black hair', 'grey hair', 'dark hair', 'blue eyes', 'blue irises', 'blue eyeballs', 'brown eyes', 'brown irises', 'brown eyeballs', 'black eyes', 'black irises', 'black eyeballs', 'red eyes', 'red irises', 'red eyeballs', 'hazel eyes', 'hazel irises', 'hazel eyeballs', 'green eyes', 'green irises', 'green eyeballs', 'eyes were green', 'eyes were black', 'eyes were brown', 'eyes were hazel', 'eyes were blue', 'eyes were grey', '5 feet', "5'", "6'", "7'", '6 feet', '7 feet', 'feet tall', 'slim', 'thin', 'thick', 'tall', 'dark skin', 'white skin', 'pale skin', 'freckles', 'tattoos', 'tattoo', 'brown skin', 'black skin', 'high cheekbones', 'wrinkles', 'wrinkled', 'full lips', 'small breasts', 'curves', 'big breasts']
    other_words = ['eyes', 'face', 'skin', 'body', 'fur', 'chin', 'cheeks', 'big wolf', 'white wolf', 'beautiful', 'ugly', 'handsome', 'cute', 'gorgeous', 'sharp', 'features', 'arms', 'bicep', 'legs', 'ass', 'breasts', 'waist', 'muscular', 'pale', 'features', 'thin', 'fangs', 'tattoos', 'teeth', 'mouth', 'young', 'old', 'blood', 'bleeding', 'gaze', 'smirk', 'smile', 'lips', 'nose', 'hands', 'jaw']
    weather_words = ['morning', 'afternoon', 'evening', 'night', 'sunrise', 'sunset', 'dawn', 'dusk', 'noon', 'midnight', 'cloudy', 'rain', 'storm', 'wind', 'sun', 'sunny', 'fog', 'foggy', 'snow', 'snowy', 'hail', 'thunder', 'lightning', 'breeze', 'chilly', 'hot', 'warm', 'cold', 'frost', 'blizzard', 'temperature', 'humid', 'dry', 'drizzle', 'pouring', 'downpour', 'mist', 'overcast']
    locations_words = ['forest', 'living room', 'dining room', 'school', 'college', 'training grounds', 'field', 'bathroom', 'bedroom', 'cabin', 'house']
   
    clothes_results = search_in_all_chapters(chapters, clothes_words, 'Clothes')
    hair_results = search_in_all_chapters(chapters, hair_words, 'Hair')
    appearances_results = search_in_all_chapters(chapters, appearances_words, 'Appearances')
    weather_results = search_in_all_chapters(chapters, weather_words, 'Weather')
    locations_results = search_in_all_chapters(chapters, locations_words, 'Locations')
    other_results = search_in_all_chapters(chapters, other_words, 'Other')

    all_results = clothes_results + hair_results + appearances_results + weather_results + locations_results + other_results

    all_data_df = pd.DataFrame(all_results, columns=['Chapter Number', 'Chapter Title', 'Match', 'Category'])
    all_data_df.sort_values(by=['Chapter Number'], inplace=True)

    os.makedirs(f"{chapters_folder}", exist_ok=True)

    excel_file_path = f"{chapters_folder}/details.xlsx"
    try:
        all_data_df.to_excel(excel_file_path, sheet_name='Details', index=False)
        print(f"Файл Excel создан: {excel_file_path}")
    except Exception as e:
        print(f"Ошибка при сохранении Excel файла: {e}")

# Папка для сохранения
chapters_folder = "million_dollar_bride"
docx_file_path = f"{chapters_folder}/fulltext.docx"
main(docx_file_path, chapters_folder)


Извлекаю текст из million_dollar_bride/fulltext.docx...
Пытаюсь открыть файл: million_dollar_bride/fulltext.docx
Файл успешно открыт.
Текст успешно извлечён.
Разделяю текст на главы...
Текст успешно разделён на 93 глав.
Ищу заданные слова для категории Clothes...
Найдено совпадений в 277 главах для категории Clothes.
Ищу заданные слова для категории Hair...
Найдено совпадений в 178 главах для категории Hair.
Ищу заданные слова для категории Appearances...
Найдено совпадений в 129 главах для категории Appearances.
Ищу заданные слова для категории Weather...
Найдено совпадений в 306 главах для категории Weather.
Ищу заданные слова для категории Locations...
Найдено совпадений в 201 главах для категории Locations.
Ищу заданные слова для категории Other...
Найдено совпадений в 2104 главах для категории Other.
Файл Excel создан: million_dollar_bride/details.xlsx


In [15]:
import os
import re
import pandas as pd

def search_words_in_chapter(chapter_text, words):
    """Ищет слова в тексте главы и возвращает их с позициями."""
    word_pattern = r'\b(' + '|'.join(re.escape(word) for word in words) + r')\b'
    matches = []

    for match in re.finditer(word_pattern, chapter_text, re.IGNORECASE):
        start, end = match.start(), match.end()

        # Собираем контексты вокруг найденного слова
        start_context = chapter_text[:start].split()[-8:]
        end_context = chapter_text[end:].split()[:8]
        found_word = chapter_text[start:end]
        result = ' '.join(start_context + [found_word] + end_context)

        # Сохраняем позицию и результат
        matches.append((start, result))

    return matches

def search_in_all_chapters(chapters, words, category):
    """Ищет слова во всех главах и подсчитывает общее количество совпадений."""
    total_matches = 0
    results = []

    for chapter_number, chapter_title, chapter_text in chapters:
        matches = search_words_in_chapter(chapter_text, words)
        total_matches += len(matches)  # Увеличиваем счетчик совпадений

        for position, match in matches:
            results.append([chapter_number, chapter_title, position, match, category])

    print(f"✅ Категория '{category}': найдено {total_matches} совпадений.")
    return results

def main(docx_file_path, chapters_folder):
    """Основная логика обработки и поиска."""
    print(f"📂 Проверка наличия файла: {docx_file_path}")
    if not os.path.exists(docx_file_path):
        print(f"❌ Файл не найден: {docx_file_path}")
        return

    print("📖 Извлечение текста из файла...")
    book_text = extract_text_from_docx(docx_file_path)
    if not book_text:
        print("❌ Не удалось извлечь текст из файла.")
        return

    print("✂️ Разделение текста на главы...")
    chapters = split_text_into_chapters(book_text)
    if not chapters:
        print("❌ Не удалось разделить текст на главы.")
        return

    # Списки ключевых слов для поиска
    clothes_words = ['stilettos', 'sunglasses', 'ring', 'necklace', 'bracelet', 'earrings', 'brooch', 'watch', 'anklet', 'choker', 'pendant', 'cufflinks', 'tie clip', 'nose ring', 'belly button ring', 'toe ring', 'hairpin', 'tiara', 'diadem', 'bangle', 'chain', 'medallion', 'pearl necklace', 'locket', 'armband', 'charm bracelet', 'dress', 'robe', 'suit', 'clothes', 'coat', 'jacket', 'shirt', 'pants', 'skirt', 'jeans', 't-shirt', 'sweater', 'blouse', 'shorts', 'hoodie', 'vest', 'scarf', 'hat', 'gloves', 'boots', 'shoes', 'sneakers', 'socks', 'tie', 'belt', 'gown', 'trench coat', 'blazer', 'cardigan', 'overalls', 'tank top', 'leggings']
    hair_words = ['hair', 'beard', 'ponytail', 'bun', 'braids', 'bob', 'pixie cut', 'long waves', 'curly hair', 'straight hair', 'afro', 'buzz cut', 'french twist', 'dreadlocks', 'fishtail braid', 'half-up half-down', 'side part', 'middle part', 'updo', 'loose curls', 'locks', 'layered cut', 'shag cut', 'crew cut', 'mohawk', 'bangs', 'chignon', 'top knot']
    appearances_words = ['fur', 'black wolf', 'white wolf', 'brown wolf', 'caramel hair', 'blonde', 'brunette', 'redhead', 'white hair', 'red hair', 'auburn hair', 'chestnut hair', 'black hair', 'grey hair', 'dark hair', 'blue eyes', 'blue irises', 'blue eyeballs', 'brown eyes', 'brown irises', 'brown eyeballs', 'black eyes', 'black irises', 'black eyeballs', 'red eyes', 'red irises', 'red eyeballs', 'hazel eyes', 'hazel irises', 'hazel eyeballs', 'green eyes', 'green irises', 'green eyeballs', 'eyes were green', 'eyes were black', 'eyes were brown', 'eyes were hazel', 'eyes were blue', 'eyes were grey', '5 feet', "5'", "6'", "7'", '6 feet', '7 feet', 'feet tall', 'slim', 'thin', 'thick', 'tall', 'dark skin', 'white skin', 'pale skin', 'freckles', 'tattoos', 'tattoo', 'brown skin', 'black skin', 'high cheekbones', 'wrinkles', 'wrinkled', 'full lips', 'small breasts', 'curves', 'big breasts']
    other_words = ['eyes', 'face', 'skin', 'body', 'fur', 'chin', 'cheeks', 'big wolf', 'white wolf', 'beautiful', 'ugly', 'handsome', 'cute', 'gorgeous', 'sharp', 'features', 'arms', 'bicep', 'legs', 'ass', 'breasts', 'waist', 'muscular', 'pale', 'features', 'thin', 'fangs', 'tattoos', 'teeth', 'mouth', 'young', 'old', 'blood', 'bleeding', 'gaze', 'smirk', 'smile', 'lips', 'nose', 'hands', 'jaw']
    weather_words = ['morning', 'afternoon', 'evening', 'night', 'sunrise', 'sunset', 'dawn', 'dusk', 'noon', 'midnight', 'cloudy', 'rain', 'storm', 'wind', 'sun', 'sunny', 'fog', 'foggy', 'snow', 'snowy', 'hail', 'thunder', 'lightning', 'breeze', 'chilly', 'hot', 'warm', 'cold', 'frost', 'blizzard', 'temperature', 'humid', 'dry', 'drizzle', 'pouring', 'downpour', 'mist', 'overcast']
    locations_words = ['forest', 'living room', 'dining room', 'school', 'college', 'training grounds', 'field', 'bathroom', 'bedroom', 'cabin', 'house']
   

    all_results = []
    for words, category in [
        (clothes_words, 'Clothes'), (hair_words, 'Hair'),
        (appearances_words, 'Appearances'), (weather_words, 'Weather'),
        (locations_words, 'Locations'), (other_words, 'Other')
    ]:
        all_results.extend(search_in_all_chapters(chapters, words, category))

    print("📊 Создание DataFrame с результатами...")
    all_data_df = pd.DataFrame(all_results, columns=['Chapter Number', 'Chapter Title', 'Position', 'Match', 'Category'])
    
    # Сортируем по номеру главы и позиции
    all_data_df.sort_values(by=['Chapter Number', 'Position'], inplace=True)

    print("💾 Сохранение результатов в Excel...")
    os.makedirs(chapters_folder, exist_ok=True)
    excel_file_path = f"{chapters_folder}/details.xlsx"
    
    # Убираем колонку 'Position' перед сохранением
    try:
        all_data_df.drop(columns=['Position']).to_excel(excel_file_path, sheet_name='Details', index=False)
        print(f"✅ Файл Excel создан: {excel_file_path}")
    except Exception as e:
        print(f"❌ Ошибка при сохранении Excel файла: {e}")

# Запуск основного процесса
chapters_folder = "million_dollar_bride"
docx_file_path = f"{chapters_folder}/fulltext.docx"
main(docx_file_path, chapters_folder)


📂 Проверка наличия файла: million_dollar_bride/fulltext.docx
📖 Извлечение текста из файла...
✂️ Разделение текста на главы...
✅ Категория 'Clothes': найдено 277 совпадений.
✅ Категория 'Hair': найдено 178 совпадений.
✅ Категория 'Appearances': найдено 129 совпадений.
✅ Категория 'Weather': найдено 306 совпадений.
✅ Категория 'Locations': найдено 201 совпадений.
✅ Категория 'Other': найдено 2104 совпадений.
📊 Создание DataFrame с результатами...
💾 Сохранение результатов в Excel...
✅ Файл Excel создан: million_dollar_bride/details.xlsx
